In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import pickle
warnings.filterwarnings('ignore')
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
#Function to reccomend song
def song_recommendation(name):
    secrets_file = open("secrets.txt","r")
    string = secrets_file.read()
    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            secrets_dict[line.split(':')[0]]=line.split(':')[1]
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                               client_secret=secrets_dict['csecret']))
    search = sp.search(q='track:' + name, type='track')  # searches for all the track info
    track_uri = search['tracks']['items'][0]['uri']      # get the track uri from search output
    features = sp.audio_features(track_uri)              # gets the audio features

    features_df = pd.DataFrame(features[0], index=[0])[['danceability', 'energy', 'loudness', 'speechiness',
                          'acousticness', 'instrumentalness', 'liveness', 
                          'valence', 'tempo']] # put features into dataframe

    scaler = pickle.load(open('scaler.pkl', 'rb'))  # load scaler 
    model = pickle.load(open('model.pkl', 'rb'))    # load model
    X_prep = scaler.transform(features_df)          # use scaler on features
    cluster = model.predict(X_prep)                 # predict cluster for that song
    clustered_df = pd.read_csv('clustered_df.csv', index_col='Unnamed: 0') #read our df with 10,000
    return clustered_df[clustered_df['cluster'] == cluster[0]].sample(1)['song'].values[0] 
            #match the cluster to the cluster in my df and give me one random song



In [3]:
song_recommendation('like a rolling stone')

'Psycho Killer - 2005 Remaster'

In [4]:
def recommendation():
    top100 = pd.read_csv('top100.csv', index_col='Unnamed: 0')
    random_song=random.choice(top100['song'])
    user_choice = input ('Choose a song from the top 100:') 
    
    if user_choice.lower() not in list(top100['song']):
        song = song_recommendation(user_choice) #applying above function here on user choice
        print("If you like the song " + "'" + user_choice + "'" + " then listen to this next: " + "'" + song + "'")
    else:
        print("If you like the song " + "'" + user_choice + "'" + " then listen to this next: " + "'" + random_song + "'")

In [5]:
recommendation()

Choose a song from the top 100:ZZZ
If you like the song 'ZZZ' then listen to this next: 'Nantes'
